In [25]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from config import mysqlpassword 
# import os

# Extract

In [2]:
path = 'Resources\salary_raw.csv'

In [3]:
# read csv file as DataFrame and display
salary_df = pd.read_csv(path, index_col=0)
salary_df.head()

,RK,Name,Team,Salary,Season
0,RK,NAME,TEAM,SALARY,2018-2019
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2018-2019
2,2,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2018-2019
3,3,"Chris Paul, PG",Houston Rockets,"$35,654,150",2018-2019
4,4,"Blake Griffin, PF",Detroit Pistons,"$32,088,932",2018-2019


# Transform

In [4]:
# Clean rows
salary_df.dropna (how='all')
salary_df = salary_df[salary_df.RK != 'RK']
salary_df.reset_index(drop=True, inplace=True)
salary_df.head()

,RK,Name,Team,Salary,Season
0,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2018-2019
1,2,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2018-2019
2,3,"Chris Paul, PG",Houston Rockets,"$35,654,150",2018-2019
3,4,"Blake Griffin, PF",Detroit Pistons,"$32,088,932",2018-2019
4,5,"Gordon Hayward, SF",Boston Celtics,"$31,214,295",2018-2019


In [5]:
# Split name from position and create new column
s = salary_df['Name'].str.split(', ', expand=True)
salary_df['Name'] = s[0]
salary_df['Position'] = s[1]
salary_df.head()

,RK,Name,Team,Salary,Season,Position
0,1,Stephen Curry,Golden State Warriors,"$37,457,154",2018-2019,PG
1,2,Russell Westbrook,Oklahoma City Thunder,"$35,654,150",2018-2019,PG
2,3,Chris Paul,Houston Rockets,"$35,654,150",2018-2019,PG
3,4,Blake Griffin,Detroit Pistons,"$32,088,932",2018-2019,PF
4,5,Gordon Hayward,Boston Celtics,"$31,214,295",2018-2019,SF


In [6]:
# Create foreign key to be used for joins based on names (index remains primary key)
salary_df['namekey'] = salary_df['Name'].str.lower()
for s in [" ", "'", "-", ".", ","]:
    salary_df['namekey'] = salary_df['namekey'].str.replace(s,'')
salary_df.head()

,RK,Name,Team,Salary,Season,Position,namekey
0,1,Stephen Curry,Golden State Warriors,"$37,457,154",2018-2019,PG,stephencurry
1,2,Russell Westbrook,Oklahoma City Thunder,"$35,654,150",2018-2019,PG,russellwestbrook
2,3,Chris Paul,Houston Rockets,"$35,654,150",2018-2019,PG,chrispaul
3,4,Blake Griffin,Detroit Pistons,"$32,088,932",2018-2019,PF,blakegriffin
4,5,Gordon Hayward,Boston Celtics,"$31,214,295",2018-2019,SF,gordonhayward


In [7]:
# Use second year of season as season
salary_df["Season"] = salary_df["Season"].str.split('-', expand=True)[1]
salary_df.head()

,RK,Name,Team,Salary,Season,Position,namekey
0,1,Stephen Curry,Golden State Warriors,"$37,457,154",2019,PG,stephencurry
1,2,Russell Westbrook,Oklahoma City Thunder,"$35,654,150",2019,PG,russellwestbrook
2,3,Chris Paul,Houston Rockets,"$35,654,150",2019,PG,chrispaul
3,4,Blake Griffin,Detroit Pistons,"$32,088,932",2019,PF,blakegriffin
4,5,Gordon Hayward,Boston Celtics,"$31,214,295",2019,SF,gordonhayward


In [8]:
# Select/reorder columns
salary_df = salary_df[['Name', 'Season', 'RK', 'Team', 'Salary', 'namekey']]
salary_df.index.name = 'id'
salary_df.head()

,Name,Season,RK,Team,Salary,namekey
id,,,,,,
0,Stephen Curry,2019,1,Golden State Warriors,"$37,457,154",stephencurry
1,Russell Westbrook,2019,2,Oklahoma City Thunder,"$35,654,150",russellwestbrook
2,Chris Paul,2019,3,Houston Rockets,"$35,654,150",chrispaul
3,Blake Griffin,2019,4,Detroit Pistons,"$32,088,932",blakegriffin
4,Gordon Hayward,2019,5,Boston Celtics,"$31,214,295",gordonhayward


# Load

In [28]:
# Create Database Connection
rds_connection_string = f"root:{mysqlpassword}@localhost/basketball_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# Confirm tables
engine.table_names()

['players', 'salary']

In [29]:
# Load Dataframe into database
salary_df.to_sql(name='salary', con=engine, if_exists='replace')

In [30]:
# Confirm data added succesfully
pd.read_sql_query('select * from salary', con=engine).head()

,id,Name,Season,RK,Team,Salary,namekey
0,0,Stephen Curry,2019,1,Golden State Warriors,"$37,457,154",stephencurry
1,1,Russell Westbrook,2019,2,Oklahoma City Thunder,"$35,654,150",russellwestbrook
2,2,Chris Paul,2019,3,Houston Rockets,"$35,654,150",chrispaul
3,3,Blake Griffin,2019,4,Detroit Pistons,"$32,088,932",blakegriffin
4,4,Gordon Hayward,2019,5,Boston Celtics,"$31,214,295",gordonhayward
